In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [2]:
df_returns = pd.read_csv('../Stocks/Returns_Data.csv')
df_stock_characteristics = pd.read_csv('../Stocks/Stock_Characteristics_Data_rating.csv')
df_time_series = pd.read_csv('../Stocks/Time_Series_Data.csv')

In [7]:
df_returns['date'] = pd.to_datetime(df_returns['date'], format='%Y%m%d')
df_returns['year_month'] = df_returns['date'].dt.to_period('M')
df_stock_characteristics['datadate'] = pd.to_datetime(df_stock_characteristics['datadate'], format='%Y%m%d')
df_stock_characteristics['year_month'] = df_stock_characteristics['datadate'].dt.to_period('M')
df_time_series['yyyymm'] = pd.to_datetime(df_time_series['yyyymm'], format='%Y%m')
df_time_series['year_month'] = df_time_series['yyyymm'].dt.to_period('M')

In [8]:
df_returns.head()

,TICKER,date,PRC,VOL,RET,vwretd,ewretd,year_month
0,ORCL,2010-01-29,23.0600,6068156.0,-0.057888,-0.037186,-0.011388,2010-01
1,ORCL,2010-02-26,24.6500,5771538.0,0.068951,0.034754,0.033603,2010-02
2,ORCL,2010-03-31,25.7100,6618577.0,0.043002,0.063688,0.069289,2010-03
3,ORCL,2010-04-30,25.8675,5580407.0,0.008071,0.020039,0.057843,2010-04
4,ORCL,2010-05-28,22.5700,7406752.0,-0.127477,-0.079267,-0.076929,2010-05


In [9]:
df_stock_characteristics.head()

,datadate,fyearq,fqtr,tic,conm,rating,atq,ceqq,cheq,chq,...,pstkrq,seqq,txditcq,xsgaq,capxy,epspxy,costat,mkvaltq,sic,year_month
0,2010-03-31,2010,1.0,AAL,AMERICAN AIRLINES GROUP INC,CCC,25525.0,-3892.0,5006.0,189.0,...,0.0,-3892.0,0.0,739.0,317.0,-1.52,A,3034.1766,4512,2010-03
1,2010-06-30,2010,2.0,AAL,AMERICAN AIRLINES GROUP INC,CCC,25885.0,-3930.0,5545.0,197.0,...,0.0,-3930.0,0.0,636.0,729.0,-1.55,A,2258.1468,4512,2010-06
2,2010-09-30,2010,3.0,AAL,AMERICAN AIRLINES GROUP INC,CCC,25357.0,-3643.0,5004.0,202.0,...,0.0,-3643.0,0.0,659.0,1412.0,-1.12,A,2088.2862,4512,2010-09
3,2010-12-31,2010,4.0,AAL,AMERICAN AIRLINES GROUP INC,CCC,25088.0,-3945.0,4946.0,168.0,...,0.0,-3945.0,NaN,695.0,1962.0,-1.41,A,2597.5755,4512,2010-12
4,2011-03-31,2011,1.0,AAL,AMERICAN AIRLINES GROUP INC,CCC,27113.0,-3949.0,6254.0,286.0,...,0.0,-3949.0,NaN,731.0,359.0,-1.31,A,2151.5676,4512,2011-03


In [10]:
df_time_series.head()

,yyyymm,Index,D12,E12,b/m,tbl,AAA,BAA,lty,ntis,Rfree,infl,ltr,corpr,svar,csp,CRSP_SPvw,CRSP_SPvwx,year_month
0,2000-01-01,1394.46,16.715333,49.096667,0.154654,0.0532,0.0778,0.0833,0.0666,0.025359,0.0041,0.002971,0.0228,-0.0021,0.005206,0.000622,-0.049630,-0.050296,2000-01
1,2000-02-01,1366.42,16.738667,50.023333,0.167056,0.0555,0.0768,0.0829,0.0646,0.027419,0.0043,0.005924,0.0264,0.0092,0.003000,0.000405,-0.017458,-0.018666,2000-02
2,2000-03-01,1498.58,16.762000,50.950000,0.149974,0.0569,0.0768,0.0837,0.0618,0.018291,0.0047,0.008245,0.0367,0.0169,0.006678,0.000185,0.098488,0.097431,2000-03
3,2000-04-01,1452.43,16.742667,51.273333,0.152600,0.0566,0.0764,0.0840,0.0630,0.011945,0.0046,0.000584,-0.0076,-0.0115,0.007942,-0.000387,-0.031585,-0.032308,2000-04
4,2000-05-01,1420.60,16.723333,51.596667,0.155669,0.0579,0.0799,0.0890,0.0640,0.009179,0.0050,0.001168,-0.0054,-0.0161,0.005185,0.000009,-0.022304,-0.023617,2000-05


In [11]:
period_range = pd.period_range(start=df_stock_characteristics['year_month'].min(), end=df_stock_characteristics['year_month'].max())
df_stock_characteristics = df_stock_characteristics.set_index('year_month').groupby('tic').apply(lambda x: x.reindex(period_range).ffill()).reset_index(0, drop=True)
df_stock_characteristics[df_stock_characteristics['datadate'].notnull()]
df_stock_characteristics.index.names = ['year_month']
df_stock_characteristics.reset_index(level=0, inplace=True)
df_stock_characteristics

,year_month,datadate,fyearq,fqtr,tic,conm,rating,atq,ceqq,cheq,...,pstkq,pstkrq,seqq,txditcq,xsgaq,capxy,epspxy,costat,mkvaltq,sic
0,2010-01,2010-01-31,2010.0,1.0,A,AGILENT TECHNOLOGIES INC,BBB,7574.0,2590.0,4046.0,...,0.0,0.0,2590.0,NaN,511.0,25.0,0.23,A,9754.4400,3826.0
1,2010-02,2010-01-31,2010.0,1.0,A,AGILENT TECHNOLOGIES INC,BBB,7574.0,2590.0,4046.0,...,0.0,0.0,2590.0,NaN,511.0,25.0,0.23,A,9754.4400,3826.0
2,2010-03,2010-01-31,2010.0,1.0,A,AGILENT TECHNOLOGIES INC,BBB,7574.0,2590.0,4046.0,...,0.0,0.0,2590.0,NaN,511.0,25.0,0.23,A,9754.4400,3826.0
3,2010-04,2010-04-30,2010.0,2.0,A,AGILENT TECHNOLOGIES INC,BBB,7767.0,2624.0,4209.0,...,0.0,0.0,2624.0,NaN,526.0,54.0,0.54,A,12618.4800,3826.0
4,2010-05,2010-04-30,2010.0,2.0,A,AGILENT TECHNOLOGIES INC,BBB,7767.0,2624.0,4209.0,...,0.0,0.0,2624.0,NaN,526.0,54.0,0.54,A,12618.4800,3826.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71495,2021-07,2021-06-30,2021.0,2.0,ZTS,ZOETIS INC,BBB-,14069.0,4350.0,3658.0,...,0.0,0.0,4350.0,381.0,615.0,190.0,2.25,A,88373.9620,2834.0
71496,2021-08,2021-06-30,2021.0,2.0,ZTS,ZOETIS INC,BBB-,14069.0,4350.0,3658.0,...,0.0,0.0,4350.0,381.0,615.0,190.0,2.25,A,88373.9620,2834.0
71497,2021-09,2021-09-30,2021.0,3.0,ZTS,ZOETIS INC,BBB-,13705.0,4679.0,3274.0,...,0.0,0.0,4679.0,291.0,636.0,309.0,3.42,A,91900.8284,2834.0
71498,2021-10,2021-09-30,2021.0,3.0,ZTS,ZOETIS INC,BBB-,13705.0,4679.0,3274.0,...,0.0,0.0,4679.0,291.0,636.0,309.0,3.42,A,91900.8284,2834.0


In [12]:
"""
datadate    	Date
fyearq	      Fiscal year
fqtr	        Fiscal quarter
tic	          Ticker/Trading symbol
conm	        Company name
atq	          Assets - Total
ceqq	        Common/ordinary equity - Total
cheq	        Cash and short-term investments
chq	          Cash
cshoq	        Common shares outstanding
dlcq	        Debt in current liabilities
dpq	          Depreciation and amortization - Total
intanq	      Intangible assets - Total
ltq	          Liabilities - Total
niq	          Net income (loss)
pstkq	        Preferred/preference stock (capital) - Total
pstkrq	      Preferred/preference stock redeemable
seqq	        Stockholders equity - Parent
txditcq	      Deferred taxes and investment tax credit
xsgaq	        Selling, general and administrative expenses
capxy	        Capital expenditures
epspxy	      Earnings per share (basic) excluding extraordinary items
costat	      Active/inactive status marker (A=active, I=inactive)
mkvaltq	      Market value - Total
sic	          Standard Industrial Classification code
"""
# Only limited data available for January and February 2010
df_fundamental_means = df_stock_characteristics[['year_month', 'ltq', 'atq', 'ceqq', 'cheq', 'chq', 'cshoq', 'dlcq', 'dpq', 'intanq', 'niq', 'pstkq', 'pstkrq', 'seqq', 'txditcq', 'xsgaq', 'capxy', 'epspxy', 'mkvaltq']].groupby('year_month').mean()[2:]
df_fundamental_means

,ltq,atq,ceqq,cheq,chq,cshoq,dlcq,dpq,intanq,niq,pstkq,pstkrq,seqq,txditcq,xsgaq,capxy,epspxy,mkvaltq
year_month,,,,,,,,,,,,,,,,,,
2010-03,42857.669302,53395.474426,9935.196492,6875.235360,1922.631206,639.825165,5674.628613,216.016167,5647.128156,346.625909,323.583339,19.895053,10279.538430,1264.091973,854.232766,244.601345,0.816826,22188.135435
2010-04,42894.744271,53440.308609,9943.008123,6886.242677,1925.004064,639.598077,5702.709940,216.039692,5658.518150,346.254400,323.579238,19.890960,10287.305339,1267.347749,853.905859,213.545743,0.793348,22407.089966
2010-05,42898.551824,53448.718457,9950.170523,6880.696345,1923.559812,639.879077,5701.623931,216.410519,5657.768272,350.214262,321.020969,19.890960,10291.925247,1263.151432,857.279322,220.109859,0.820783,22371.079445
2010-06,42031.515836,52739.715952,10114.141769,7050.737974,1949.446109,639.613007,5451.678648,216.054940,5574.205625,394.803737,310.582580,19.328308,10445.680794,1271.374806,876.852296,399.750486,1.423341,20163.198227
2010-07,42060.989562,52768.188614,10112.677345,7053.795485,1950.275248,638.991952,5465.586723,216.258548,5586.087075,393.667158,310.991547,19.324603,10444.597731,1268.560516,877.250425,413.800283,1.429806,19946.370395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07,68049.332234,86081.137320,17173.511750,12950.168264,4169.988790,637.801558,4500.307628,412.658331,11539.705833,936.234014,416.031744,10.279916,17589.543494,1779.181602,1341.237802,735.572110,4.291080,77563.073933
2021-08,68033.805702,86058.506190,17167.163548,12928.196576,4151.948152,637.641722,4484.584514,413.281179,11541.413801,935.729258,416.031744,10.279916,17583.195292,1779.768505,1346.222190,732.412864,4.313800,77955.764893
2021-09,68973.345992,87243.438506,17406.545280,13304.172880,4150.770265,620.410402,4479.005196,415.304237,11740.825149,907.885958,422.989338,8.968558,17829.534618,1778.849153,1362.256298,1004.561554,6.117820,77891.455113


In [13]:
# Target variable to predict
macro_target = 'AAA'

In [14]:
df_macro = df_time_series[['year_month', macro_target]]
df_macro = df_macro.set_index('year_month')
df_macro

,AAA
year_month,
2000-01,0.0778
2000-02,0.0768
2000-03,0.0768
2000-04,0.0764
2000-05,0.0799
...,...
2020-08,0.0225
2020-09,0.0231
2020-10,0.0235


In [15]:
df_data = df_macro.merge(df_fundamental_means.shift(1), left_index=True, right_index=True)
df_data = df_data.tail(df_data.shape[0] - 1)
df_data

,AAA,ltq,atq,ceqq,cheq,chq,cshoq,dlcq,dpq,intanq,niq,pstkq,pstkrq,seqq,txditcq,xsgaq,capxy,epspxy,mkvaltq
year_month,,,,,,,,,,,,,,,,,,,
2010-04,0.0529,42857.669302,53395.474426,9935.196492,6875.235360,1922.631206,639.825165,5674.628613,216.016167,5647.128156,346.625909,323.583339,19.895053,10279.538430,1264.091973,854.232766,244.601345,0.816826,22188.135435
2010-05,0.0496,42894.744271,53440.308609,9943.008123,6886.242677,1925.004064,639.598077,5702.709940,216.039692,5658.518150,346.254400,323.579238,19.890960,10287.305339,1267.347749,853.905859,213.545743,0.793348,22407.089966
2010-06,0.0488,42898.551824,53448.718457,9950.170523,6880.696345,1923.559812,639.879077,5701.623931,216.410519,5657.768272,350.214262,321.020969,19.890960,10291.925247,1263.151432,857.279322,220.109859,0.820783,22371.079445
2010-07,0.0472,42031.515836,52739.715952,10114.141769,7050.737974,1949.446109,639.613007,5451.678648,216.054940,5574.205625,394.803737,310.582580,19.328308,10445.680794,1271.374806,876.852296,399.750486,1.423341,20163.198227
2010-08,0.0449,42060.989562,52768.188614,10112.677345,7053.795485,1950.275248,638.991952,5465.586723,216.258548,5586.087075,393.667158,310.991547,19.324603,10444.597731,1268.560516,877.250425,413.800283,1.429806,19946.370395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08,0.0225,62833.552140,79305.943982,15632.932506,12573.206428,4132.332831,595.917499,4422.720326,397.899778,11100.280233,340.231394,414.420330,14.158062,16047.352836,1652.465910,1174.979700,700.781194,1.875200,55542.359575
2020-09,0.0231,62855.186510,79341.706472,15647.141788,12590.589672,4128.117321,596.035170,4418.002712,398.753145,11101.712361,359.818472,414.420330,14.158062,16061.562118,1652.562886,1178.139395,694.094804,1.932940,55912.569928
2020-10,0.0235,63171.821302,79949.969048,15931.950050,12376.204084,4021.608213,623.401433,4231.963818,386.467419,11153.461616,607.331622,416.102630,13.195704,16348.052680,1654.887172,1208.994648,927.440786,3.046080,60018.263935


In [16]:
df_X = df_data[df_fundamental_means.columns]
df_y = df_data[df_macro.columns]

In [17]:
df_X

,ltq,atq,ceqq,cheq,chq,cshoq,dlcq,dpq,intanq,niq,pstkq,pstkrq,seqq,txditcq,xsgaq,capxy,epspxy,mkvaltq
year_month,,,,,,,,,,,,,,,,,,
2010-04,42857.669302,53395.474426,9935.196492,6875.235360,1922.631206,639.825165,5674.628613,216.016167,5647.128156,346.625909,323.583339,19.895053,10279.538430,1264.091973,854.232766,244.601345,0.816826,22188.135435
2010-05,42894.744271,53440.308609,9943.008123,6886.242677,1925.004064,639.598077,5702.709940,216.039692,5658.518150,346.254400,323.579238,19.890960,10287.305339,1267.347749,853.905859,213.545743,0.793348,22407.089966
2010-06,42898.551824,53448.718457,9950.170523,6880.696345,1923.559812,639.879077,5701.623931,216.410519,5657.768272,350.214262,321.020969,19.890960,10291.925247,1263.151432,857.279322,220.109859,0.820783,22371.079445
2010-07,42031.515836,52739.715952,10114.141769,7050.737974,1949.446109,639.613007,5451.678648,216.054940,5574.205625,394.803737,310.582580,19.328308,10445.680794,1271.374806,876.852296,399.750486,1.423341,20163.198227
2010-08,42060.989562,52768.188614,10112.677345,7053.795485,1950.275248,638.991952,5465.586723,216.258548,5586.087075,393.667158,310.991547,19.324603,10444.597731,1268.560516,877.250425,413.800283,1.429806,19946.370395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08,62833.552140,79305.943982,15632.932506,12573.206428,4132.332831,595.917499,4422.720326,397.899778,11100.280233,340.231394,414.420330,14.158062,16047.352836,1652.465910,1174.979700,700.781194,1.875200,55542.359575
2020-09,62855.186510,79341.706472,15647.141788,12590.589672,4128.117321,596.035170,4418.002712,398.753145,11101.712361,359.818472,414.420330,14.158062,16061.562118,1652.562886,1178.139395,694.094804,1.932940,55912.569928
2020-10,63171.821302,79949.969048,15931.950050,12376.204084,4021.608213,623.401433,4231.963818,386.467419,11153.461616,607.331622,416.102630,13.195704,16348.052680,1654.887172,1208.994648,927.440786,3.046080,60018.263935


In [18]:
df_y

,AAA
year_month,
2010-04,0.0529
2010-05,0.0496
2010-06,0.0488
2010-07,0.0472
2010-08,0.0449
...,...
2020-08,0.0225
2020-09,0.0231
2020-10,0.0235
